## Análisis por slices problemáticos del modelo BETO (Sprint 4)

Objetivo:
Analizar el comportamiento de BETO en subgrupos específicos (“slices”) del conjunto de prueba, tal como se solicitó en el Sprint 4, para identificar patrones de error o posibles sesgos.


1️⃣ Imports y configuración

Celda de código:

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

Mapa de etiquetas (igual que en todos):

In [2]:
id2label = {
    0: "ad_hominem",
    1: "framing_binario",
    2: "logos",
    3: "retorica_vacia",
}
label2id = {v: k for k, v in id2label.items()}
id2label


{0: 'ad_hominem', 1: 'framing_binario', 2: 'logos', 3: 'retorica_vacia'}

2️⃣ Cargar modelo final y datasets 

2.1. Modelo BETO final

In [3]:
MODEL_DIR = "./models/beto_v3_final_fold0"  # el que guardaste en el 3

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.to(device)
model.eval()

print("Modelo cargado desde:", MODEL_DIR)


Modelo cargado desde: ./models/beto_v3_final_fold0


2.2. Dataset con folds (para usar el Fold 4 como test interno)

In [4]:
PATH_FOLDS = "../data/processed/clean_v2/folds/corpus_clean_v2_folds.csv"
df_full = pd.read_csv(PATH_FOLDS)

df_full.head()


,text,labels,group_id,fold
0,Habla de verdad y solo cuenta su parte,0,3026b3a2a78a2b7c351ce65e1c8a1aea,2
1,La fuerza del pueblo es más fuerte que cualqui...,3,584e534c015f8b6bd23d2d3b57d74283,1
2,Más del 60 % de los empleos creados en pandemi...,2,ba640e8244144484ba43f58ba7b84688,1
3,No hay fuerza más grande que la esperanza,3,deb5caa3e7be895393e0b2e0be670727,0
4,Se llena la boca de patria y vacía los bolsill...,0,261fc2fef1f426ea0a7a90e882286225,2


Elegimos Fold 4 como test:

In [5]:
TEST_FOLD = 4

df_test = df_full[df_full["fold"] == TEST_FOLD].reset_index(drop=True)
print("Tamaño test Fold 4:", len(df_test))
df_test[["text", "labels", "fold"]].head()


Tamaño test Fold 4: 198


,text,labels,fold
0,Más de 50 000 escolares abandonaron clases tra...,2,4
1,El 45 % de los jóvenes peruanos trabaja sin co...,2,4
2,[ADVERSARIO] almuerzan en restaurantes de lujo...,1,4
3,"[ADVERSARIO] nos dividen, [PARTIDO] seguimos r...",1,4
4,"[PARTIDO] ponemos los votos, [ADVERSARIO] pone...",1,4


2.3. Hold-out humano (opcional pero recomendado)

In [6]:
df_holdout = pd.read_csv("../data/holdout_humano.csv")
df_holdout.head()


,text,label_name,label
0,"Dice ser honesto, pero encubre a sus amigos",ad_hominem,0
1,"Quieren destruir el país, nosotros vamos a res...",framing_binario,1
2,El 40% de los jóvenes perdió su empleo el últi...,logos,2
3,"Promete cambio, pero no explica cómo lo hará",retorica_vacia,3


3️⃣ Función de inferencia por lotes

Igual que en los otros notebooks:

In [7]:
def predict_batch(texts, batch_size=32):
    all_pred_ids = []
    all_probs = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        enc = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt",
        )
        enc = {k: v.to(device) for k, v in enc.items()}

        with torch.no_grad():
            outputs = model(**enc)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=-1).cpu().numpy()

        pred_ids = np.argmax(probs, axis=1)
        all_pred_ids.extend(pred_ids.tolist())
        all_probs.append(probs)

    all_probs = np.vstack(all_probs)
    pred_labels = [id2label[int(i)] for i in all_pred_ids]
    return np.array(all_pred_ids), pred_labels, all_probs


4️⃣ Predicciones base (Fold 4 y hold-out humano)

4.1. Fold 4 (interno)

In [8]:
texts_test = df_test["text"].tolist()
true_ids_test = df_test["labels"].tolist()

pred_ids_test, pred_labels_test, probs_test = predict_batch(texts_test, batch_size=32)

df_test_pred = df_test.copy()
df_test_pred["true_label_id"] = true_ids_test
df_test_pred["true_label_name"] = df_test_pred["labels"].map(id2label)
df_test_pred["pred_label_id"] = pred_ids_test
df_test_pred["pred_label_name"] = pred_labels_test

print("Ejemplos Fold 4 con predicciones:")
df_test_pred[["text", "true_label_name", "pred_label_name"]].head()


Ejemplos Fold 4 con predicciones:


,text,true_label_name,pred_label_name
0,Más de 50 000 escolares abandonaron clases tra...,logos,logos
1,El 45 % de los jóvenes peruanos trabaja sin co...,logos,logos
2,[ADVERSARIO] almuerzan en restaurantes de lujo...,framing_binario,framing_binario
3,"[ADVERSARIO] nos dividen, [PARTIDO] seguimos r...",framing_binario,framing_binario
4,"[PARTIDO] ponemos los votos, [ADVERSARIO] pone...",framing_binario,framing_binario


4.2. Hold-out humano (externo)

In [9]:
texts_h = df_holdout["text"].tolist()
true_ids_h = df_holdout["label"].tolist()

pred_ids_h, pred_labels_h, probs_h = predict_batch(texts_h, batch_size=32)

df_holdout["true_label_id"] = true_ids_h
df_holdout["true_label_name"] = df_holdout["label_name"]
df_holdout["pred_label_id"] = pred_ids_h
df_holdout["pred_label_name"] = pred_labels_h

print("Ejemplos hold-out con predicciones:")
df_holdout[["text", "true_label_name", "pred_label_name"]].head()


Ejemplos hold-out con predicciones:


,text,true_label_name,pred_label_name
0,"Dice ser honesto, pero encubre a sus amigos",ad_hominem,ad_hominem
1,"Quieren destruir el país, nosotros vamos a res...",framing_binario,retorica_vacia
2,El 40% de los jóvenes perdió su empleo el últi...,logos,logos
3,"Promete cambio, pero no explica cómo lo hará",retorica_vacia,ad_hominem


5️⃣ Slice 1 – Textos ultracortos (≤ 5 palabras)

Primero definimos longitud (usaremos el hold-out humano que es donde tienes errores):

In [10]:
df_holdout["num_tokens"] = df_holdout["text"].str.split().apply(len)

slice_ultrashort = df_holdout[df_holdout["num_tokens"] <= 5]

if len(slice_ultrashort) > 0:
    f1_ultrashort = f1_score(
        slice_ultrashort["true_label_id"],
        slice_ultrashort["pred_label_id"],
        average="macro",
    )
else:
    f1_ultrashort = float("nan")

print("Slice: textos ultracortos (<=5 palabras)")
print("F1 macro:", f1_ultrashort)
print("Tamaño del slice:", len(slice_ultrashort))
slice_ultrashort[["text", "true_label_name", "pred_label_name", "num_tokens"]].head()


Slice: textos ultracortos (<=5 palabras)
F1 macro: nan
Tamaño del slice: 0


,text,true_label_name,pred_label_name,num_tokens


6️⃣ Slice 2 – Frases en primera persona (yo/me/mi/nosotros/nos)

In [11]:
person_terms = [" yo ", " me ", " mi ", " nosotros ", " nos "]

df_holdout["has_1p"] = df_holdout["text"].str.lower().apply(
    lambda x: any(t in " " + x + " " for t in person_terms)
)

slice_1p = df_holdout[df_holdout["has_1p"] == True]

if len(slice_1p) > 0:
    f1_1p = f1_score(
        slice_1p["true_label_id"],
        slice_1p["pred_label_id"],
        average="macro",
    )
else:
    f1_1p = float("nan")

print("Slice: frases con primera persona (yo/me/mi/nosotros/nos)")
print("F1 macro:", f1_1p)
print("Tamaño del slice:", len(slice_1p))
slice_1p[["text", "true_label_name", "pred_label_name"]].head()


Slice: frases con primera persona (yo/me/mi/nosotros/nos)
F1 macro: 0.0
Tamaño del slice: 1


,text,true_label_name,pred_label_name
1,"Quieren destruir el país, nosotros vamos a res...",framing_binario,retorica_vacia


7️⃣ Slice 3 (opcional) – Frases con números (típico de logos)

Este es otro slice chévere para el profe:

In [12]:
df_holdout["has_number"] = df_holdout["text"].str.contains(r"\d", regex=True)

slice_num = df_holdout[df_holdout["has_number"] == True]

if len(slice_num) > 0:
    f1_num = f1_score(
        slice_num["true_label_id"],
        slice_num["pred_label_id"],
        average="macro",
    )
else:
    f1_num = float("nan")

print("Slice: frases con números (potencialmente logos)")
print("F1 macro:", f1_num)
print("Tamaño del slice:", len(slice_num))
slice_num[["text", "true_label_name", "pred_label_name"]].head()


Slice: frases con números (potencialmente logos)
F1 macro: 1.0
Tamaño del slice: 1


,text,true_label_name,pred_label_name
2,El 40% de los jóvenes perdió su empleo el últi...,logos,logos


8️⃣ Resumen de slices (como mini tabla final)

In [13]:
resumen_slices = pd.DataFrame([
    {
        "Slice": "Textos ultracortos (<=5 palabras)",
        "F1_macro": f1_ultrashort,
        "Tamano": len(slice_ultrashort),
    },
    {
        "Slice": "Frases con primera persona",
        "F1_macro": f1_1p,
        "Tamano": len(slice_1p),
    },
    {
        "Slice": "Frases con números",
        "F1_macro": f1_num,
        "Tamano": len(slice_num),
    },
])

resumen_slices


,Slice,F1_macro,Tamano
0,Textos ultracortos (<=5 palabras),NaN,0
1,Frases con primera persona,0.0,1
2,Frases con números,1.0,1


## Conclusiones de los slices problemáticos (Sprint 4)

- **Textos ultracortos (≤ 5 palabras):**  
  [Comentar el F1 obtenido y si es menor o similar al global. Si hay pocos ejemplos,
  aclarar que el tamaño del slice es reducido.]

- **Frases en primera persona (yo/me/mi/nosotros/nos):**  
  [Comentar si el modelo mantiene un buen desempeño o si se confunde más cuando
  el discurso es más subjetivo y autorreferencial.]

- **Frases con números (potencialmente logos):**  
  [Normalmente el F1 será muy alto, porque el modelo reconoce bien los patrones
  asociados a argumentos basados en cifras.]

En conjunto, estos slices permiten observar que el modelo BETO se comporta de forma
robusta en la mayoría de subgrupos, pero que ciertas configuraciones retóricas
(textos muy cortos, uso de primera persona, etc.) pueden acercarse a las zonas
donde se producen las confusiones observadas en el hold-out humano
(framework binario ↔ retórica vacía ↔ ad hominem).
